In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Drugs.com)/drugsComTest_raw.csv
/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Drugs.com)/drugsComTrain_raw.csv
/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Druglib.com)/drugLibTest_raw.csv
/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Druglib.com)/drugLibTrain_raw.csv


In [2]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem import SnowballStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
import string
import re
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve,f1_score

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = pd.read_csv('/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Drugs.com)/drugsComTrain_raw.csv')
test = pd.read_csv('/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Drugs.com)/drugsComTest_raw.csv')

In [4]:
print(train.shape)
print(test.shape)

(161297, 7)
(53766, 7)


In [5]:
train.columns

Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   161297 non-null  int64  
 1   drugName     161297 non-null  object 
 2   condition    160398 non-null  object 
 3   review       161297 non-null  object 
 4   rating       161297 non-null  float64
 5   date         161297 non-null  object 
 6   usefulCount  161297 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 8.6+ MB


In [7]:
len(train)

161297

In [8]:
train.isna().sum()/len(train)*100

Unnamed: 0     0.000000
drugName       0.000000
condition      0.557357
review         0.000000
rating         0.000000
date           0.000000
usefulCount    0.000000
dtype: float64

In [9]:
train = train.drop('Unnamed: 0',axis=1)

In [10]:
train.head()

,drugName,condition,review,rating,date,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [11]:
df_train = train.dropna(how='any')
df_test = train.dropna(how='any')

In [12]:
top_20_conditions = pd.DataFrame(df_train['condition'].value_counts()).reset_index()
px.bar(top_20_conditions[:50],x='condition',y='count')

In [13]:
index_counts = df_train['condition'].value_counts()[df_train['condition'].value_counts()>=20].index
df_train = df_train[df_train['condition'].isin(index_counts)]


In [14]:
# balancing classes
condition_over_200 = df_train['condition'].value_counts()[df_train['condition'].value_counts()>=200].index
for condition in condition_over_200:
    #shuffling the samples
    condition_samples = df_train[df_train['condition']== condition]
    condition_samples= condition_samples.sample(frac=1).reset_index(drop=True)
#     print(condition_samples)
    # extract only 200
    condition_samples = condition_samples[:200]
    df_train =df_train[df_train['condition']!=condition]
    df_train = pd.concat([df_train,condition_samples],ignore_index=True)

In [15]:
df_train

,drugName,condition,review,rating,date,usefulCount
0,Pentasa,Crohn's Disease,"""I had Crohn&#039;s with a resection 30 years ...",4.0,"July 6, 2013",8
1,Nilotinib,Chronic Myelogenous Leukemia,"""I have been on Tasigna for just over 3 years ...",10.0,"September 1, 2015",11
2,Eflornithine,Hirsutism,"""I&#039;m writing a second review on Vaniqa. ...",10.0,"May 11, 2014",99
3,Trilafon,Psychosis,"""I had a similar experience. Tremors in hands...",9.0,"May 23, 2011",45
4,Opdivo,Non-Small Cell Lung Cance,"""My mother died from lung cancer. Her last hop...",1.0,"March 2, 2017",6
...,...,...,...,...,...,...
36354,Dramamine,Motion Sickness,"""I enjoy sport fishing. most of the time about...",9.0,"August 17, 2016",5
36355,Transderm-Scop,Motion Sickness,"""These patches are a miracle. I get motion si...",10.0,"November 12, 2008",40
36356,Transderm-Scop,Motion Sickness,"""I have suffered from motion sickness my entir...",8.0,"November 30, 2015",12
36357,Scopolamine,Motion Sickness,"""I took this for a 6 hour car ride followed up...",8.0,"March 17, 2015",10


In [16]:
def filter_labels(labels):
    labels = labels.tolist()
    labels_truth = []
    for label in labels:
        if label[0].isdigit():
            labels_truth.append(False)
        else:
            labels_truth.append(True)
    return labels_truth

In [17]:
# sum(filter_labels(df_train['condition']))

In [18]:
df_train = df_train[filter_labels(df_train['condition'])]

In [19]:
df_test = df_test[filter_labels(df_test['condition'])]

In [20]:
print(df_train.condition.nunique())
print(df_test.condition.nunique())

318
811


In [21]:
df_test = df_test[df_test['condition'].isin(df_train['condition'])]

In [22]:
print(df_test.condition.nunique())

318


In [23]:
df_train.loc[:,'review'] = df_train['review'].str.lower()
df_test.loc[:,'review'] = df_test['review'].str.lower()

In [24]:
def new_filter_data(reviews):
    stop = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')

    def process_text(text):
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove special characters
        text = re.sub(r'\W+', ' ', text)
        # Remove stopwords
        text = ' '.join([word for word in text.split() if word.lower() not in stop])
        # Stem words
        text = ' '.join([stemmer.stem(word) for word in text.split()])
        return text

    if isinstance(reviews, pd.Series):
        reviews = reviews.apply(process_text)
    elif isinstance(reviews, str):
        reviews = process_text(reviews)
    else:
        raise TypeError("Input should be a string or a pandas Series of strings.")

    return reviews

df_train['review'] = new_filter_data(df_train['review'])
df_test['review'] = new_filter_data(df_test['review'])

/tmp/ipykernel_34/653374317.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
df_train['review'][0]

'crohn039 resect 30 year ago most remiss sinc recent bad flare narrow anastomosi need medic haven039t found one handl asacol gave serious bodi ach fatigu could function pentasa immedi gave heart palpit arrhythmia discontinu'

In [26]:
df_train['label'] = df_train['condition'].str.lower()
df_test['label'] = df_test['condition'].str.lower()

/tmp/ipykernel_34/367633204.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
df_test['label']

1                                 adhd
2                        birth control
3                        birth control
4                    opiate dependence
5         benign prostatic hyperplasia
                      ...             
161292              alcohol dependence
161293                 nausea/vomiting
161294            rheumatoid arthritis
161295             underactive thyroid
161296           constipation, chronic
Name: label, Length: 156827, dtype: object

In [28]:
df_train = df_train.sample(frac=1)
X = df_train['review']
y = df_train['label']

In [29]:
count_vect = CountVectorizer(ngram_range=(1,2))
X_cv = count_vect.fit_transform(X)

In [30]:
def cross_val_multiple_classifiers(X,y):
    classifiers = [MultinomialNB(), SGDClassifier(loss="modified_huber"), 
                 RandomForestClassifier(n_estimators=100), 
                 KNeighborsClassifier(n_neighbors=5)]
    
    labels = ['Multinomial Naive Bayes', 'SGD Classifier', 'Random Forest',  'KNN']
    clf_cv_mean = []
    clf_cv_std = []
    for clf,label in zip(classifiers,labels):
        scores = cross_val_score(clf, X, y, cv=4, scoring='accuracy')
        print("Accuracy : %.2f (+/- %.2f) [%s]" %(scores.mean(),scores.std(),label))

In [31]:
cross_val_multiple_classifiers(X_cv,y)

Accuracy : 0.49 (+/- 0.01) [Multinomial Naive Bayes]
Accuracy : 0.55 (+/- 0.00) [SGD Classifier]
Accuracy : 0.61 (+/- 0.00) [Random Forest]
Accuracy : 0.07 (+/- 0.01) [KNN]


In [32]:
final_clf = RandomForestClassifier(n_estimators=100)
final_clf.fit(X_cv,y)
X_test_vec = count_vect.transform(df_test['review'])
preds = final_clf.predict(X_test_vec)

In [33]:
preds

array(['adhd', 'abnormal uterine bleeding', 'menstrual disorders', ...,
       'rosacea', "hashimoto's disease", 'constipation, chronic'],
      dtype=object)

In [34]:
print('Accuracy on test data: ',accuracy_score(df_test['label'].str.lower(),preds))
print('Macro F1 Score on test data: ',f1_score(df_test['label'].str.lower(),preds,average='macro'))
print('Micro F1 Score on test data: ',f1_score(df_test['label'].str.lower(),preds,average='micro'))

Accuracy on test data:  0.5884063330931536
Macro F1 Score on test data:  0.7480156014412275
Micro F1 Score on test data:  0.5884063330931536


In [35]:
print(classification_report(df_test['label'],preds))

                                                            precision    recall  f1-score   support

                                 abnormal uterine bleeding       0.20      0.68      0.30      2096
                                            acial wrinkles       0.72      0.97      0.83        32
                                                      acne       0.68      0.84      0.75      5588
                                         actinic keratosis       0.85      1.00      0.92        29
                                   acute coronary syndrome       0.72      0.96      0.83        54
                                                      adhd       0.84      0.75      0.79      3383
                                            agitated state       0.81      1.00      0.89        38
                                        alcohol dependence       0.45      0.95      0.61       388
                                        alcohol withdrawal       0.30      0.99      0.45       217

In [36]:
drug_lib_ts = pd.read_csv('/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Druglib.com)/drugLibTest_raw.csv')

In [37]:
drug_lib_ts.columns

Index(['Unnamed: 0', 'urlDrugName', 'rating', 'effectiveness', 'sideEffects',
       'condition', 'benefitsReview', 'sideEffectsReview', 'commentsReview'],
      dtype='object')

In [49]:
sample = new_filter_data(drug_lib_ts['commentsReview'][20])
sample_cv = count_vect.transform([sample])
final_clf.predict(sample_cv)

array(['psoriasis'], dtype=object)

In [51]:
drug_lib_ts[['commentsReview','condition']].loc[20]

commentsReview    I started with 4 tabs and increased to 6 as th...
condition                                       psoraitic arthritis
Name: 20, dtype: object